<a href="https://colab.research.google.com/github/jvishnuvardhan/TF_Lite/blob/master/tflite_Rnn_ConcreteFun_SavedModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tf-nightly

     |████████████████████████████████| 522.4MB 28kB/s 
     |████████████████████████████████| 51kB 7.3MB/s 
     |████████████████████████████████| 3.0MB 49.3MB/s 
     |████████████████████████████████| 460kB 41.0MB/s 
  Found existing installation: Keras-Preprocessing 1.1.0
    Uninstalling Keras-Preprocessing-1.1.0:
      Successfully uninstalled Keras-Preprocessing-1.1.0


In [2]:
import tensorflow as tf

x_inputs = tf.keras.layers.Input((3,229,1))
x = tf.keras.layers.Conv2D(32, (3,3), padding='same')(x_inputs)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.Dropout(0.25)(x)
x = tf.keras.layers.Conv2D(64, (3,3), padding='same')(x)
x = tf.keras.layers.Reshape((-1,tf.keras.backend.int_shape(x)[-1] * tf.keras.backend.int_shape(x)[-2]))(x)
x = tf.keras.layers.LSTM(256)(x)
x = tf.keras.layers.Dense(1)(x)
x = tf.keras.layers.Activation('sigmoid')(x)


model = tf.keras.models.Model(x_inputs, x)
model.summary()

@tf.function
def call(x):
  return model(x)

concrete_func = call.get_concrete_function(
    x = tf.TensorSpec(shape=[3,229,1], dtype=tf.float32))
export_dir = '/tmp/saved_model'
model.save(export_dir, save_format="tf", signatures=concrete_func)

converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)
tflite_model = converter.convert()

with open('model.tflite', 'wb') as fp:
  fp.write(tflite_model)

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 3, 229, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 3, 229, 32)        320       
_________________________________________________________________
batch_normalization (BatchNo (None, 3, 229, 32)        128       
_________________________________________________________________
activation (Activation)      (None, 3, 229, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 3, 229, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 3, 229, 64)        18496     
_________________________________________________________________
reshape (Reshape)            (None, None, 14656)      

In [3]:
import numpy as np
test_img = tf.random.uniform((3,229,1),dtype=tf.float32)
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()
input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]

# Pre-processing: add batch dimension and convert to float32 to match with
# the model's input data format.
#test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
interpreter.set_tensor(input_index, test_img)

# Run inference.
interpreter.invoke()

# Post-processing: remove batch dimension and find the digit with highest
# probability.
output = interpreter.tensor(output_index)
out = np.argmax(output()[0])
print(out)

0
